In [ ]:

import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input/agedata'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten
import kerastuner as kt

In [ ]:
mydata= pd.read_csv("../input/agedata/age_gender.csv")
mydata.drop("img_name",axis=1, inplace= True)
mydata.head(10)

In [ ]:
count_age=mydata[mydata["age"]==26]["age"].count()
len_age=len(mydata["age"])
print("There are {} labelled data for age 26 and length of age data is {}".format(count_age,len_age))

In [ ]:
x_gender= mydata.iloc[:,-1]

In [ ]:
x_age=[]
y_age=[]
j=0
for i in range(len(mydata["age"])):
    if mydata["age"][i]==26:
        if j<1200:
            j+=1
            x_age.append(mydata["pixels"][i])
            y_age.append(mydata["age"][i])
    elif mydata["age"][i]>70:
        continue
    else:
        x_age.append(mydata["pixels"][i])
        y_age.append(mydata["age"][i])
        

In [ ]:
img_data_age=[]
for i in x_age:
    image=np.array(i.split(" "),dtype="float32")
    image=image.reshape(48,48,1)
    img_data_age.append(image)
    
img_data_age=np.array(img_data_age)
img_data_age/=255
img_data_age= img_data_age.reshape(img_data_age.shape[0],48,48,1)

y_age= np.array(y_age, dtype="int")

In [ ]:
xtrain_age, xtest_age, ytrain_age, ytest_age= train_test_split(img_data_age, y_age, test_size=0.33, random_state=38)


In [ ]:
stop_early= keras.callbacks.EarlyStopping(monitor="val_loss",patience=2)

In [ ]:
def model_builder_age(hp):
    image_input= keras.Input(shape=(48,48,1))
    x= Conv2D(16, kernel_size=3, activation="relu")(image_input)
    x= Conv2D(32, kernel_size=3, activation="relu")(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    x= Dropout(0.30)(x)
    x= Conv2D(64, kernel_size=3, activation="relu")(x)
    x= Conv2D(96, kernel_size=3, activation="relu")(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    x= Dropout(0.30)(x)
    x= Conv2D(128, kernel_size=3, activation="relu")(x)
    x= Conv2D(160, kernel_size=3, activation="relu")(x)
    x= BatchNormalization()(x)
    x= Flatten()(x)
    x= Dense(units=256, activation= "relu")(x)
    image_output= Dense(1)(x)
    model= keras.Model(image_input, image_output)
    hp_losses= hp.Choice("loss_type", values=["mean_absolute_percentage_error","mean_absolute_error"])
    if hp_losses=="mean_absolute_error":
        model.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-3),
                loss= "mean_absolute_error",
                metrics=["mean_squared_error"])
    else:
        model.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-2),
                loss= "mean_squared_error",
                metrics=["mean_squared_error"])
        
    return model

In [ ]:
tuner_age = kt.Hyperband(model_builder_age,
                     objective='val_loss',
                     max_epochs=5,
                     factor=3,
                     directory='my_dir',
                     project_name='face_age_age')

In [ ]:
tuner_age.search(xtrain_age, ytrain_age , epochs=5, validation_split=0.2, callbacks=[stop_early])
best_hps_age=tuner_age.get_best_hyperparameters(num_trials=1)[0]

print(f""" best loss {best_hps_age.get('loss_type')}. """)


In [ ]:
model_age= tuner_age.hypermodel.build(best_hps_age)
history_age= model_age.fit(xtrain_age, ytrain_age, epochs=10, validation_split= 0.2)

val_loss_per_epoch_age=history_age.history["val_loss"]
best_epoch_age= val_loss_per_epoch_age.index(max(val_loss_per_epoch_age))+1

print('Best epoch for age: %d' % (best_epoch_age,))

In [ ]:
hypermodel_age = tuner_age.hypermodel.build(best_hps_age)
hypermodel_age.fit(xtest_age, ytest_age, epochs=best_epoch_age)

In [ ]:
eval_result_age = hypermodel_age.evaluate(xtest_age, ytest_age)

In [ ]:
print("[test loss for age, test accuracy for age]:", eval_result_age)